## Mobile Customer Churn Prediction
Customer churn refers to the loss of existing clients or customers. This solution identifies Mobile customers who are more likely to close their account and leave the Mobile Company. During the training stage, the solution automatically conducts feature interaction on the training data and selects a subset of features based on feature importance. It then trains multiple models and identifies the best performing model. This model is then selected for prediction on new data.

### Contents

1. [Set up the environment](#Set-up-the-environment)
1. [Usage Instructions](#Usage-Instructions)
1. [Upload the data for training](#Upload-the-data-for-training)
1. [Run Training Job](#Run-Training-Job)
1. [Live Inference Endpoint](#Live Inference)
1. [Batch Transform Job](#Batch-Transform-Job)
1. [Output Interpretation](#Output-Interpretation)



<img src="images/Flow_diagram.JPG">

### Prerequisite

To run this algorithm you need to have access to the following AWS Services:
- Access to AWS SageMaker and the model package.
- An S3 bucket to specify input/output.
- Role for AWS SageMaker to access input/output from S3.

### Input format
#### Input:
Name of the file: <b>train.csv</b><br>
This file contains historical churn prediction data for Ecommerce. T<br><br>

</ul>
<li>  Tenure:Tenure of customer in organization</li>
<li> PreferredLoginDevice: Preferred login device of customer</li>

<li> CityTier: City tier</li>
<li> WarehouseToHome: Distance in between warehouse to home of customer</li>
<li> PreferredPaymentMode: Preferred payment method of customer</li>
<li> Gender: Gender of customer</li>
<li> HourSpendOnApp: Number of hours spend on mobile application or website</li>
    
<li> NumberOfDeviceRegistered: Total number of deceives is registered on particular customer</li>
<li> PreferedOrderCat: Preferred order category of customer in last month</li>
<li> SatisfactionScore: Satisfactory score of customer on service</li>
<li> MaritalStatus: Marital status of customer</li>
<li> NumberOfAddress: Total number of added added on particular customer
    
 <li>Complain: Any complaint has been raised in last month</li>
<li> OrderAmountHikeFromlastYear: Percentage increases in order from last year</li>
<li> CouponUsed: Total number of coupon has been used in last month</li>
<li> OrderCount: Total number of orders has been places in last month</li>
<li> DaySinceLastOrder: Day Since last order by customer</li>
    






## Set up the environment
Here we specify a bucket to use and the role that will be used for working with SageMaker.

In [10]:
# S3 prefix
prefix = 'churn-mobile'

# Define IAM role
import boto3
import re

import os
import numpy as np
import pandas as pd
from sagemaker import get_execution_role

role = get_execution_role()

## Create the session
The session remembers our connection parameters to SageMaker. We'll use it to perform all of our SageMaker operations.

In [11]:
import sagemaker as sage
from time import gmtime, strftime

sess = sage.Session()

## Upload the data for training
When training large models with huge amounts of data, you'll typically use big data tools, like Amazon Athena, AWS Glue, or Amazon EMR, to create your data in S3. For the purposes of this example, we're using classification dataset, which we have included.

We can use use the tools provided by the SageMaker Python SDK to upload the data to a default bucket.

In [12]:
data_location= 's3://mphasis-marketplace/churn-prediction-mobile/input/train.csv'

## Create an estimator and fit the model
In order to use SageMaker to fit our algorithm, we'll create an Estimator that defines how to use the container to train. This includes the configuration we need to invoke SageMaker training:
- The container name. This is constructed as in the shell commands above.
- The role. As defined above.
- The instance count which is the number of machines to use for training.
- The instance type which is the type of machine to use for training.
- The output path determines where the model artifact will be written.
- The session is the SageMaker session object that we defined above

Then we use fit() on the estimator to train against the data that we uploaded above.

In [13]:
account = sess.boto_session.client('sts').get_caller_identity()['Account']
region = sess.boto_session.region_name
image = '{}.dkr.ecr.{}.amazonaws.com/telecom-pycaret-churn'.format(account, region)

tree = sage.estimator.Estimator(image,
                       role, 3, 'ml.c4.2xlarge',
                      output_path="s3://{}/output".format(sess.default_bucket()),
                       sagemaker_session=sess)

tree.fit(data_location)

2021-04-28 08:44:16 Starting - Starting the training job...
2021-04-28 08:44:20 Starting - Launching requested ML instancesProfilerReport-1619599456: InProgress
......
2021-04-28 08:45:35 Starting - Preparing the instances for training......
2021-04-28 08:46:35 Downloading - Downloading input data
2021-04-28 08:46:35 Training - Downloading the training image......
2021-04-28 08:47:35 Training - Training image download completed. Training in progress.Starting the training.
(66469, 66)
IntProgress(value=0, description='Processing: ', max=3)
                                                                    
                                                                    
Initiated  . . . . . . . . . . . . . . . . . .              08:47:31
Status     . . . . . . . . . . . . . . . . . .  Loading Dependencies
                                                                           
                                                                           
Initiated  . . . . . . . . 

## Hosting your model
You can use a trained model to get real time predictions using HTTP endpoint. Follow these steps to walk you through the process.


In [14]:
training_job_name = tree.latest_training_job.name
attached_tree = sage.estimator.Estimator.attach(training_job_name)



2021-04-28 09:09:00 Starting - Preparing the instances for training
2021-04-28 09:09:00 Downloading - Downloading input data
2021-04-28 09:09:00 Training - Training image download completed. Training in progress.
2021-04-28 09:09:00 Uploading - Uploading generated training model
2021-04-28 09:09:00 Completed - Training job completed



### Deploy the model
Deploying the model to SageMaker hosting just requires a deploy call on the fitted model. This call takes an instance count, instance type, and optionally serializer and deserializer functions. These are used when the resulting predictor is created on the endpoint.

In [ ]:

from sagemaker.predictor import csv_serializer
predictor = attached_tree.deploy(4, 'ml.m4.xlarge', serializer=csv_serializer,endpoint_name='churn-mobile-pycaret')

--------------!

## Choose some data and use it for a prediction


In [16]:
test_data  = 's3://mphasis-marketplace/churn-prediction-mobile/input/test.csv'

data = pd.read_csv(test_data,encoding='ISO-8859–1',header=None)
data

,0,1,2,3,4,5,6,7,8,9,...,55,56,57,58,59,60,61,62,63,64
0,year,month,user_account_id,user_lifetime,user_intake,user_no_outgoing_activity_in_days,user_account_balance_last,user_spendings,user_has_outgoing_calls,user_has_outgoing_sms,...,last_100_reloads_sum,last_100_calls_outgoing_duration,last_100_calls_outgoing_to_onnet_duration,last_100_calls_outgoing_to_offnet_duration,last_100_calls_outgoing_to_abroad_duration,last_100_sms_outgoing_count,last_100_sms_outgoing_to_onnet_count,last_100_sms_outgoing_to_offnet_count,last_100_sms_outgoing_to_abroad_count,last_100_gprs_usage
1,2013,6,13,1000,0,1,0.05,0,1,1,...,12.01,75.27,0,63.43,0,210,1,84,0,0
2,2013,6,14,1000,0,25,28.31,3.45,1,0,...,0,13.38,11.18,2,11.18,0,0,0,0,0
3,2013,6,18,1005,0,8,15.62,1.97,1,0,...,0,30,0,0,10.45,0,0,0,0,0
4,2013,6,27,1013,0,11,5.62,0,1,0,...,0,0,0,0,0,0,0,0,0,0
5,2013,6,32,1032,0,2,5.86,0.15,1,0,...,6,2.58,0,1,0,0,0,0,0,0
6,2013,6,35,1039,0,1,12.51,12.96,1,1,...,29,356.65,0,215.83,0,4,1,1,0,0
7,2013,6,36,1040,0,1,16.73,2.34,1,1,...,12,0,0,0,0,112,0,109,0,0
8,2013,6,47,521,0,1,0.66,45.89,1,1,...,158,735.87,0,473.5,0,1123,0,1006,0,0
9,2013,6,52,824,0,2,4.49,1.37,1,1,...,30.03,56.22,0,48.02,0,122,1,1,0,2.77


In [17]:
predictions = predictor.predict(data.values).decode('utf-8')



The csv_serializer has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.


In [18]:
print(predictions)

year,month,user_account_id,user_lifetime,user_intake,user_no_outgoing_activity_in_days,user_account_balance_last,user_spendings,user_has_outgoing_calls,user_has_outgoing_sms,user_use_gprs,user_does_reload,reloads_inactive_days,reloads_count,reloads_sum,calls_outgoing_count,calls_outgoing_spendings,calls_outgoing_duration,calls_outgoing_spendings_max,calls_outgoing_duration_max,calls_outgoing_inactive_days,calls_outgoing_to_onnet_count,calls_outgoing_to_onnet_spendings,calls_outgoing_to_onnet_duration,calls_outgoing_to_onnet_inactive_days,calls_outgoing_to_offnet_count,calls_outgoing_to_offnet_spendings,calls_outgoing_to_offnet_duration,calls_outgoing_to_offnet_inactive_days,calls_outgoing_to_abroad_count,calls_outgoing_to_abroad_spendings,calls_outgoing_to_abroad_duration,calls_outgoing_to_abroad_inactive_days,sms_outgoing_count,sms_outgoing_spendings,sms_outgoing_spendings_max,sms_outgoing_inactive_days,sms_outgoing_to_onnet_count,sms_outgoing_to_onnet_spendings,sms_outgoing_to_onnet_

### Output

Output files contains column predicted Group, which has the predicted class

In [19]:
transform_output_folder = "batch-transform-output"
output_path="s3://{}/{}".format(sess.default_bucket(), transform_output_folder)

transformer = tree.transformer(instance_count=1,
                               instance_type='ml.m4.xlarge',
                               output_path=output_path)

In [20]:
transformer.transform(test_data, content_type='text/csv')
transformer.wait()
print("Batch Transform output saved to " + transformer.output_path)

..................................Starting the inference server with 4 workers.
[2021-04-28 09:32:57 +0000] [12] [INFO] Starting gunicorn 20.1.0
[2021-04-28 09:32:57 +0000] [12] [INFO] Listening at: unix:/tmp/gunicorn.sock (12)
[2021-04-28 09:32:57 +0000] [12] [INFO] Using worker: gevent
[2021-04-28 09:32:57 +0000] [16] [INFO] Booting worker with pid: 16
[2021-04-28 09:32:58 +0000] [17] [INFO] Booting worker with pid: 17
[2021-04-28 09:32:58 +0000] [18] [INFO] Booting worker with pid: 18
[2021-04-28 09:32:58 +0000] [23] [INFO] Booting worker with pid: 23
Transformation Pipeline and Model Successfully Loaded
169.254.255.130 - - [28/Apr/2021:09:33:07 +0000] "GET /ping HTTP/1.1" 200 1 "-" "Go-http-client/1.1"
169.254.255.130 - - [28/Apr/2021:09:33:07 +0000] "GET /execution-parameters HTTP/1.1" 404 2 "-" "Go-http-client/1.1"
Transformation Pipeline and Model Successfully Loaded
169.254.255.130 - - [28/Apr/2021:09:33:08 +0000] "POST /invocations HTTP/1.1" 200 3721 "-" "Go-http-client/1.1"
2

#### Inspect the Batch Transform Output in S3

In [21]:
from urllib.parse import urlparse

parsed_url = urlparse(transformer.output_path)
bucket_name = parsed_url.netloc
file_key = '{}/{}.out'.format(parsed_url.path[1:], "test.csv")



s3_client = sess.boto_session.client('s3')

response = s3_client.get_object(Bucket = sess.default_bucket(), Key = file_key)
response_bytes = response['Body'].read().decode('utf-8')
print(response_bytes)

year,month,user_account_id,user_lifetime,user_intake,user_no_outgoing_activity_in_days,user_account_balance_last,user_spendings,user_has_outgoing_calls,user_has_outgoing_sms,user_use_gprs,user_does_reload,reloads_inactive_days,reloads_count,reloads_sum,calls_outgoing_count,calls_outgoing_spendings,calls_outgoing_duration,calls_outgoing_spendings_max,calls_outgoing_duration_max,calls_outgoing_inactive_days,calls_outgoing_to_onnet_count,calls_outgoing_to_onnet_spendings,calls_outgoing_to_onnet_duration,calls_outgoing_to_onnet_inactive_days,calls_outgoing_to_offnet_count,calls_outgoing_to_offnet_spendings,calls_outgoing_to_offnet_duration,calls_outgoing_to_offnet_inactive_days,calls_outgoing_to_abroad_count,calls_outgoing_to_abroad_spendings,calls_outgoing_to_abroad_duration,calls_outgoing_to_abroad_inactive_days,sms_outgoing_count,sms_outgoing_spendings,sms_outgoing_spendings_max,sms_outgoing_inactive_days,sms_outgoing_to_onnet_count,sms_outgoing_to_onnet_spendings,sms_outgoing_to_onnet_

### View Output
Lets read results of above transform job from s3 files and print output

In [22]:
s3_client = sess.boto_session.client('s3')
s3_client.download_file(sess.default_bucket(), "{}/test.csv.out".format(transform_output_folder), '/tmp/test.csv.out')
with open('/tmp/test.csv.out') as f:
    results = f.readlines() 
##print("Transform results: \n{}".format(''.join(results)))
string_final = ''.join(results)

print(string_final)

with open("Output.txt", "w") as text_file:
    text_file.write(string_final)

year,month,user_account_id,user_lifetime,user_intake,user_no_outgoing_activity_in_days,user_account_balance_last,user_spendings,user_has_outgoing_calls,user_has_outgoing_sms,user_use_gprs,user_does_reload,reloads_inactive_days,reloads_count,reloads_sum,calls_outgoing_count,calls_outgoing_spendings,calls_outgoing_duration,calls_outgoing_spendings_max,calls_outgoing_duration_max,calls_outgoing_inactive_days,calls_outgoing_to_onnet_count,calls_outgoing_to_onnet_spendings,calls_outgoing_to_onnet_duration,calls_outgoing_to_onnet_inactive_days,calls_outgoing_to_offnet_count,calls_outgoing_to_offnet_spendings,calls_outgoing_to_offnet_duration,calls_outgoing_to_offnet_inactive_days,calls_outgoing_to_abroad_count,calls_outgoing_to_abroad_spendings,calls_outgoing_to_abroad_duration,calls_outgoing_to_abroad_inactive_days,sms_outgoing_count,sms_outgoing_spendings,sms_outgoing_spendings_max,sms_outgoing_inactive_days,sms_outgoing_to_onnet_count,sms_outgoing_to_onnet_spendings,sms_outgoing_to_onnet_